###### Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [304]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,f1_score
from sklearn.pipeline import Pipeline
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [305]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [306]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [307]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


**Cleaning**

**Portfolio Data**

In [308]:
# Converting channels into categories, dropping email since this is common to all channels
portfolio = portfolio.join(portfolio.channels.str.join('|').str.get_dummies())
portfolio.drop(['channels', 'email'], axis=1, inplace=True)

In [309]:
portfolio.head()

,reward,difficulty,duration,offer_type,id,mobile,social,web
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,0
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,0,1
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,1
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,0,1


In [310]:
# Changing column order 
portfolio = portfolio[['id', 'difficulty', 'reward', 'duration', 'offer_type', 
                       'mobile', 'web', 'social']]

In [311]:
portfolio.head()

,id,difficulty,reward,duration,offer_type,mobile,web,social
0,ae264e3637204a6fb9bb56bc8210ddfd,10,10,7,bogo,1,0,1
1,4d5c57ea9a6940dd891ad53e9dbe8da0,10,10,5,bogo,1,1,1
2,3f207df678b143eea3cee63160fa8bed,0,0,4,informational,1,1,0
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,bogo,1,1,0
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,20,5,10,discount,0,1,0


In [312]:
# Get dummies for offer_type and removing original column
portfolio = portfolio.join(pd.get_dummies(portfolio.offer_type))
portfolio.drop(['offer_type'], axis=1, inplace=True)

In [313]:
portfolio.head()

,id,difficulty,reward,duration,mobile,web,social,bogo,discount,informational
0,ae264e3637204a6fb9bb56bc8210ddfd,10,10,7,1,0,1,1,0,0
1,4d5c57ea9a6940dd891ad53e9dbe8da0,10,10,5,1,1,1,1,0,0
2,3f207df678b143eea3cee63160fa8bed,0,0,4,1,1,0,0,0,1
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,1,1,0,1,0,0
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,20,5,10,0,1,0,0,1,0


**2. Profile Data**

In [314]:
# getting dummies for gender, unknown "None" becomes baseline
profile = profile.join(pd.get_dummies(profile.gender))
profile.drop(['gender'], axis=1, inplace=True)

In [315]:
profile.head()

,age,id,became_member_on,income,F,M,O
0,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,0,0,0
1,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0,1,0,0
2,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN,0,0,0
3,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,1,0,0
4,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN,0,0,0


In [316]:
# converting to datetime
profile.became_member_on = pd.to_datetime(profile.became_member_on, format='%Y%m%d')

In [317]:
# rearranging column order
profile = profile[['id', 'age', 'income', 'became_member_on', 'F', 'M', 'O']]

In [318]:
# renaming id column to person
profile.rename(columns={'id': 'person'}, inplace=True)

In [319]:
profile.head()

,person,age,income,became_member_on,F,M,O
0,68be06ca386d4c31939f3a4f0e3dd783,118,NaN,2017-02-12,0,0,0
1,0610b486422d4921ae7d2bf64640c50b,55,112000.0,2017-07-15,1,0,0
2,38fe809add3b4fcf9315a9694bb96ff5,118,NaN,2018-07-12,0,0,0
3,78afa995795e4d85b5d9ceeca43f5fef,75,100000.0,2017-05-09,1,0,0
4,a03223e636434f42ac4c3df47e8bac43,118,NaN,2017-08-04,0,0,0


**3. Transcript Data**

In [157]:
#merge profile and transcript
#transcript = transcript.merge(profile, on='person')
#transcript.head()

In [320]:
# Splitting out value column into two columns
transcript = transcript.join(pd.DataFrame(list(transcript.value)))
transcript.drop('value', axis=1, inplace=True)

In [321]:
# filling NaNs as blank strings across two column version of offer id
transcript['offer id'] = transcript['offer id'].fillna(value="")

In [322]:
transcript.head()

,person,event,time,offer id,amount,offer_id,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,NaN


In [323]:
# concatenating offer id fields since raw data had differing naming conventions
transcript['offer_id'] = transcript['offer id'].map(str) + transcript.offer_id.map(str)

In [324]:
# remove redundant offer id column
transcript.drop('offer id', axis=1, inplace=True)

In [325]:
transcript.rename(columns={"person": "customer_id"}, inplace = True)

In [326]:
transcript.head()

,customer_id,event,time,amount,offer_id,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9nan,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7nan,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,NaN,2906b810c7d4411798c6938adc9daaa5nan,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,NaN,fafdcd668e3743c1bb461111dcafc2a4nan,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0nan,NaN


In [327]:
len(transcript)

306534

In [328]:
transcript['event'].value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

In [329]:
offers = transcript[transcript.event.isin(['offer received', 'offer viewed', 'offer completed'])]
transactions = transcript[transcript.event == 'transaction']

In [330]:
offers.head()

,customer_id,event,time,amount,offer_id,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9nan,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7nan,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,NaN,2906b810c7d4411798c6938adc9daaa5nan,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,NaN,fafdcd668e3743c1bb461111dcafc2a4nan,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0nan,NaN


In [331]:
transactions.head()

,customer_id,event,time,amount,offer_id,reward
12654,02c083884c7d45b39cc68e1314fec56c,transaction,0,0.83,nan,NaN
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,0,34.56,nan,NaN
12659,54890f68699049c2a04d415abc25e717,transaction,0,13.23,nan,NaN
12670,b2f1cd155b864803ad8334cdf13c4bd2,transaction,0,19.51,nan,NaN
12671,fe97aa22dd3e48c8b143116a8403dd52,transaction,0,18.97,nan,NaN


In [332]:
# Dropping columns
offers.drop(['amount'], axis=1, inplace = True)
transactions.drop(['event', 'offer_id'], axis=1, inplace = True)
offers.head()

C:\Users\shenl\Anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,customer_id,event,time,offer_id,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9nan,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7nan,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5nan,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4nan,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0nan,NaN


In [333]:
transactions.head()

,customer_id,time,amount,reward
12654,02c083884c7d45b39cc68e1314fec56c,0,0.83,NaN
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,NaN
12659,54890f68699049c2a04d415abc25e717,0,13.23,NaN
12670,b2f1cd155b864803ad8334cdf13c4bd2,0,19.51,NaN
12671,fe97aa22dd3e48c8b143116a8403dd52,0,18.97,NaN


In [334]:
#get dummies for events and append to transcript
events = pd.get_dummies(transcript['event'])
events.head()

,offer completed,offer received,offer viewed,transaction
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0


In [335]:
events.rename(columns={"offer completed": "completed", "offer received": "received", "offer viewed": "viewed"}, inplace = True)

In [336]:
offers = pd.concat([offers, events], axis =1)
offers.head()

,customer_id,event,time,offer_id,reward,completed,received,viewed,transaction
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0.0,9b98b8c7a33c4b65b9aebfe6a799e6d9nan,NaN,0,1,0,0
1,a03223e636434f42ac4c3df47e8bac43,offer received,0.0,0b1e1539f2cc45b7b9fa7c272da2e1d7nan,NaN,0,1,0,0
2,e2127556f4f64592b11af22de27a7932,offer received,0.0,2906b810c7d4411798c6938adc9daaa5nan,NaN,0,1,0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0.0,fafdcd668e3743c1bb461111dcafc2a4nan,NaN,0,1,0,0
4,68617ca6246f4fbc85e91a2a49552598,offer received,0.0,4d5c57ea9a6940dd891ad53e9dbe8da0nan,NaN,0,1,0,0


In [337]:
offers.drop(['event'],axis =1, inplace = True)
offers.head()

,customer_id,time,offer_id,reward,completed,received,viewed,transaction
0,78afa995795e4d85b5d9ceeca43f5fef,0.0,9b98b8c7a33c4b65b9aebfe6a799e6d9nan,NaN,0,1,0,0
1,a03223e636434f42ac4c3df47e8bac43,0.0,0b1e1539f2cc45b7b9fa7c272da2e1d7nan,NaN,0,1,0,0
2,e2127556f4f64592b11af22de27a7932,0.0,2906b810c7d4411798c6938adc9daaa5nan,NaN,0,1,0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0.0,fafdcd668e3743c1bb461111dcafc2a4nan,NaN,0,1,0,0
4,68617ca6246f4fbc85e91a2a49552598,0.0,4d5c57ea9a6940dd891ad53e9dbe8da0nan,NaN,0,1,0,0


In [338]:
offers.viewed.value_counts()

0    248809
1     57725
Name: viewed, dtype: int64

In [339]:
#convert time to days from hours
offers['time_hour'] = offers['time']/24
transactions['time_hour'] = transactions['time']/24

<ipython-input-339-ec3a95165933>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions['time_hour'] = transactions['time']/24


In [149]:
#transcript.drop(['time'], axis = 1, inplace = True)

In [150]:
#transcript.head()

In [340]:
offers.drop(['time'], axis = 1, inplace = True)
transactions.drop(['time'], axis = 1, inplace = True)

C:\Users\shenl\Anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [341]:
offers.head()


,customer_id,offer_id,reward,completed,received,viewed,transaction,time_hour
0,78afa995795e4d85b5d9ceeca43f5fef,9b98b8c7a33c4b65b9aebfe6a799e6d9nan,NaN,0,1,0,0,0.0
1,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7nan,NaN,0,1,0,0,0.0
2,e2127556f4f64592b11af22de27a7932,2906b810c7d4411798c6938adc9daaa5nan,NaN,0,1,0,0,0.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,fafdcd668e3743c1bb461111dcafc2a4nan,NaN,0,1,0,0,0.0
4,68617ca6246f4fbc85e91a2a49552598,4d5c57ea9a6940dd891ad53e9dbe8da0nan,NaN,0,1,0,0,0.0


In [342]:
transactions.head()

,customer_id,amount,reward,time_hour
12654,02c083884c7d45b39cc68e1314fec56c,0.83,NaN,0.0
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,34.56,NaN,0.0
12659,54890f68699049c2a04d415abc25e717,13.23,NaN,0.0
12670,b2f1cd155b864803ad8334cdf13c4bd2,19.51,NaN,0.0
12671,fe97aa22dd3e48c8b143116a8403dd52,18.97,NaN,0.0


In [343]:
def get_success(customer_id, portfolio, profile, offers, transactions):
    """
    determine whether the offer was successfully redeemed by the customer.
    Input:
        customer_id: id of customer
        portfolio: portfolio dataframe
        profile: profile dataframe
        offers: dataframe with offer type and offer id.
        transactions: dataframe with transactions and amount
    Output:
        rows: list of dictionaries of successful offers.
    """
    
    customer = profile[profile['id'] == customer_id]
    
    customer_offer = offers[offers['customer_id'] == customer_id]
    customer_offer = customer_offer.drop(columns = 'customer_id').reset_index(drop = True)
    customer_trans = transactions[transactions['customer_id'] == customer_id]
    customer_trans = customer_trans.drop(columns = 'customer_id').reset_index(drop = True)
    
    received = customer_offer[customer_offer['received'] == 1].drop(['completed', 'received', 'viewed'], axis = 1).reset_index(drop=True)
    viewed = customer_offer[customer_offer['viewed'] == 1].drop(['completed', 'received', 'viewed'], axis = 1).reset_index(drop=True)
    completed = customer_offer[customer_offer['completed'] == 1].drop(['completed', 'received', 'viewed'], axis = 1).reset_index(drop=True)
    
    rows = []
    for idx in range(len(received)):

        # Initialize the current offer id
        cur_offer_id = received.iloc[idx]['offer_id']

        # Find current offer in portfolio
        cur_offer = portfolio.loc[portfolio['offer_id'] == cur_offer_id]
        duration = cur_offer['duration'].values[0]

        # Time period when offer is valid
        startime = received.iloc[idx]['time_hour']

        endtime = received.iloc[idx]['time_hour'] + duration

        # Boolean array that falls in valid time window
        valid_transaction = np.logical_and(customer_trans['time_hour'] >=startime,
                           customer_trans['time_hour'] <= endtime)

        # Boolean array for completion of offer within time window
        valid_completed = np.logical_and(completed['time_hour'] >= startime,
                           completed['time_hour'] <= endtime)

        # Boolean array for viewing of offer within time window
        valid_viewed = np.logical_and(viewed['time_hour'] >= startime,
                           viewed['time_hour'] <= endtime)

        # Determine whether the current offer was successful
        success = valid_completed.sum() > 0 and valid_viewed.sum() > 0

        # Relevant customer transactions are ones that occurred within the current offer window
        relevant_transactions = customer_trans[valid_transaction]

        # Dictionary which describes the current customers offer
        cur_row = {'offerid': cur_offer_id,
                   'customerid': customer_id,
                   'time': startime,
                   'success': int(success),
                   'totalamount': relevant_transactions['amount'].sum()}

        cur_row.update(cur_offer.iloc[0,1:].to_dict())

        cur_row.update(customer.iloc[0,1:].to_dict())

        # Update a list of dictionaries that describes the success of an offer to the customer
        rows.append(cur_row)

    return rows

In [344]:
# Test to see if function works
x = '68be06ca386d4c31939f3a4f0e3dd783'
test = pd.DataFrame(get_success(x, portfolio, profile, offers, transactions))
test.head()

KeyError: 'id'